In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from datetime import timedelta

from lib.data.dataset import Dataset
from lib.scraper.scraper import Scraper
from lib.scraper.ticker import CommonTickers
from lib.data.feature_generator import FeatureGenerator
from lib.data.features.common_features import CommonFeatures
from lib.data.features.common_targets import CommonTargets

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


First, we need to scrape the data from IEX cloud. This is done via the Scraper class.

In [2]:
s = Scraper(
    config_file='lib/scraper/config/iexcloud-sandbox-private.json',
    should_print=False
)

s

Next, we need to actually scrape the data.


*Note: In this example, we will use sandbox data from IEXCloud. This data is random and does not reflect actual stock data. Therefor, this example is for demonstration purposes only.*

In [3]:
disney_data, disney_filename = s.get_intraday_stock_data(
    ticker=CommonTickers.DISNEY,
    start='2021-04-20',
    end='2021-04-29',
    time_delta=timedelta(days=1),
    save_data=True
)

disney_data.head()

,minute,label,high,low,average,volume,notional,numberOfTrades,marketHigh,marketLow,marketAverage,marketVolume,marketNotional,marketNumberOfTrades,open,close,marketOpen,marketClose,changeOverTime,marketChangeOverTime
date,,,,,,,,,,,,,,,,,,,,
2021-04-20,09:30,09:30 AM,194.550,187.393,195.423,3606,663927.138,92,192.13,191.500,193.288,202651,3.742857e+07,211,191.790,191.440,194.33,193.640,0.000000,0.000000
2021-04-20,09:31,09:31 AM,193.480,191.740,196.159,582,106573.470,12,189.18,193.926,188.780,34240,6.383953e+06,123,196.730,187.590,190.73,189.859,-0.000486,0.001917
2021-04-20,09:32,09:32 AM,190.179,192.010,192.846,211,37880.130,7,194.74,192.390,187.120,25793,4.952529e+06,144,193.784,190.467,190.50,196.130,-0.003824,-0.001401
2021-04-20,09:33,09:33 AM,191.540,186.920,196.008,912,167921.870,10,187.41,188.890,196.065,12507,2.388168e+06,98,188.060,193.160,191.33,192.730,-0.004421,-0.002486
2021-04-20,09:34,09:34 AM,191.810,186.750,193.991,536,101135.640,9,187.63,193.930,188.021,9751,1.829149e+06,74,195.430,192.690,187.08,193.100,-0.004533,-0.002895


Now, we need to generate some features for our data and clean it up a bit:

In [4]:
feature_generator = FeatureGenerator(
    filename=disney_filename,
    auto_clean=True,
    parse_dates=True
)

feature_generator

In [5]:
feature_generator.build_features(
    [
        CommonFeatures.Sinify('day_of_year', period=365),
        CommonFeatures.Cosify('day_of_year', period=365),
        CommonFeatures.Sinify('minute_of_day', period=(60 * 24)),
        CommonFeatures.Cosify('minute_of_day', period=(60 * 24)),
        CommonFeatures.OneHotEncoder('weekday'),
        CommonFeatures.OneHotEncoder('hour_of_day')
    ]
)

Now, let's add some targets to our data that we will try to predict!

In [6]:
feature_generator.build_features(
    [
        CommonTargets.FutureValue(feature='marketLow', target_time_delta=timedelta(minutes=1)),
        CommonTargets.FutureValueChange(feature='future_value')  # This is the feature generated directly above
    ]

)

Now that we have our features generated, we can generate a dataset from our features:

In [7]:
# We don't want to include the original value of the one-hot-encoded variables
features_to_exclude = ['weekday', 'hour_of_day']
exported_data = feature_generator.export(
    target_feature='future_value_change',
    features_to_exclude=features_to_exclude
)
exported_data.head()

,marketHigh,marketLow,marketAverage,marketVolume,marketNotional,marketNumberOfTrades,marketOpen,marketClose,marketChangeOverTime,year,...,weekday_Tuesday,weekday_Wednesday,hour_of_day_10,hour_of_day_11,hour_of_day_12,hour_of_day_13,hour_of_day_14,hour_of_day_15,future_value,future_value_change
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-04-20 09:30:00,192.13,191.500,193.288,202651.0,3.742857e+07,211.0,194.33,193.640,0.000000,2021.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,193.926,-0.007921
2021-04-20 09:31:00,189.18,193.926,188.780,34240.0,6.383953e+06,123.0,190.73,189.859,0.001917,2021.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,192.390,-0.018192
2021-04-20 09:32:00,194.74,192.390,187.120,25793.0,4.952529e+06,144.0,190.50,196.130,-0.001401,2021.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,188.890,0.026682
2021-04-20 09:33:00,187.41,188.890,196.065,12507.0,2.388168e+06,98.0,191.33,192.730,-0.002486,2021.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,193.930,0.007941
2021-04-20 09:34:00,187.63,193.930,188.021,9751.0,1.829149e+06,74.0,187.08,193.100,-0.002895,2021.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,195.470,-0.043536


Now, we will instantiate an instance of the Dataset class, which manages normalization, train/test split, and saving/loading for us:

In [8]:
dataset = Dataset(
    df=exported_data,
    train_fraction=0.7,
    target_max_threshold=float('inf')
)

dataset.train.shape

(1842, 60, 29)

We can easily save this dataset to the disk for later use:

In [9]:
data_path = dataset.save_to_disk(CommonTickers.ATT.ticker)

Successfully saved dataset to `exported_data/datasets/1619751670-T/*`


And we can reload that dataset easily, like this:

In [10]:
dataset_from_disk = Dataset(folder_path=data_path)

For a sanity check, let's make sure all of the data was saved/retrieved losslessly:

In [11]:
np.array_equal(dataset.train, dataset_from_disk.train)

True

In [12]:
np.array_equal(dataset.val, dataset_from_disk.val)

True

In [13]:
np.array_equal(dataset.test, dataset_from_disk.test)

True

Now that we have all of our data loaded, let's train an example model!

In [14]:
def compile_and_fit(model, ds, patience=5, epochs=50):

    model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

    history = model.fit(ds.train_X, ds.train_y, epochs=epochs,
                      validation_data=(ds.val_X, ds.val_y))
    return history

In [15]:
dense_model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=60, activation='relu'),
    tf.keras.layers.Dense(units=1)
])
history = compile_and_fit(dense_model, dataset, 5999, 10)

Epoch 1/10
58/58 [==============================] - 1s 4ms/step - loss: 29064660992.0000 - mean_absolute_error: 84980.5234 - val_loss: 245446912.0000 - val_mean_absolute_error: 12688.5312
Epoch 2/10
58/58 [==============================] - 0s 1ms/step - loss: 82341312.0000 - mean_absolute_error: 4118.1958 - val_loss: 457151.8750 - val_mean_absolute_error: 669.4212
Epoch 3/10
58/58 [==============================] - 0s 1ms/step - loss: 366786.4375 - mean_absolute_error: 487.9862 - val_loss: 156218.1250 - val_mean_absolute_error: 375.1725
Epoch 4/10
58/58 [==============================] - 0s 1ms/step - loss: 154551.1719 - mean_absolute_error: 344.6838 - val_loss: 149573.0781 - val_mean_absolute_error: 364.7361
Epoch 5/10
58/58 [==============================] - 0s 1ms/step - loss: 151343.8125 - mean_absolute_error: 334.1245 - val_loss: 154520.5312 - val_mean_absolute_error: 373.6835
Epoch 6/10
58/58 [==============================] - 0s 1ms/step - loss: 149884.0000 - mean_absolute_error

Obviously this model sucks lol since this is completely random data we are working with (because we pulled from sandbox). BUT the loss does decrease with each epoch, so something must be working right!

Feel free to mess around with the library and create your own models!